## 1. Load the model and Test
- Before begining, `model_data_argu` dict() is written by hand for setting property

In [1]:
# Intialize the tensorflow-gpu <-> physical matching
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [20]:
from tensorflow.keras.models import load_model
model_load = load_model("./ckpt/"+"OOP-7-0.92.h5")

In [3]:
##############################3 Insert model and Data Properties
### be careful when calculating test accuracy, make sure labels_map is same as train phase and test phase(here)
model_data_argu = {'label': "OOP", 'dsize': (64,64), 'model_name': "model_oop_cnn", "labels_map": {'c5': 0, 'c0': 1, 'c6': 2, 'c1': 3, 'c7': 4}}

In [4]:
# Load Model (Test)
import utils.xai_viz
from utils.xai_viz import explainable_model
import importlib
from model.model_cnn import model_cnn
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline
import random
from load.load_data import load_mydata

myloader = load_mydata()
X_my_test, Y_my_test = myloader.load_test_data(classifier_label=model_data_argu['label'], dsize=model_data_argu['dsize'], comp_ratio=4)
print("My Test set is ", len(Y_my_test))

X_test = np.array(X_my_test)/255.0
labels_map=model_data_argu["labels_map"]

Y_my_test = np.array(list(map(lambda x: labels_map[x], Y_my_test)))
Y_test = to_categorical(np.array(Y_my_test))
print(labels_map)

My Test data folder lists are..
20211012_center_belt_nomask_minseok1
20211012_center_unbelt_nomask_minseok1
20211015_center_belt_mask_yukhyun1
20211015_center_unbelt_mask_sungwook1
20211015_center_unbelt_mask_yukhyun1
20211015_center_unbelt_nomask_sungwook1
20211015_center_unbelt_nomask_sungwook2
20211015_center_unbelt_nomask_yukhyun1
20211015_close_unbelt_nomask_sungwook1
20211017_center_belt_mask_yukhyun1
20211017_center_belt_nomask_yukhyun1
20211017_center_unbelt_mask_sungwook1
20211017_center_unbelt_nomask_sungwook1
20211019_center_belt_mask_juwon1
20211019_center_belt_mask_juwon2
20211019_center_belt_mask_juwon3
20211019_center_belt_nomask_juwon1
20211019_center_belt_nomask_juwon2
20211019_center_unbelt_mask_juwon3
20211019_center_unbelt_nomask_juwon1
20211019_close_belt_mask_juwon1
20211019_close_unbelt_mask_juwon1
20211019_far_belt_mask_juwon1
20211019_far_unbelt_mask_juwon1
20211019_phone_belt_mask_juwon1
20211019_phone_unbelt_mask_juwon1
20211020_behind_belt_nomask_sujin1
2021

Test data Loading with OOP: 2000it [00:02, 870.00it/s]

My Test set is  359
{'c5': 0, 'c0': 1, 'c6': 2, 'c1': 3, 'c7': 4}


In [5]:
print(X_test.shape, Y_test.shape)

(359, 64, 64, 3) (359, 5)


In [6]:

xai = explainable_model(model_load)

In [ ]:
X_pred=model_load.predict(X_test)
# Test using X_test data
def score(preds, labels):
    ret=0
    for pred, label in zip(preds, labels):

        pred_idx = np.argmax(pred)
        label_idx = np.argmax(label)

        if (pred_idx == label_idx):
            ret+=1

    ret = ret / len(preds)
    return ret

print("Test Predict is: {}%".format(score(X_pred, Y_test)*100))

In [ ]:
#2. (모델2: OOP + phoning**) 기존 오픈데이터셋의 라벨링 활용 가능 safe driving(c0), reaching behind(c7)
#                           ,phoning(c2,c4), texting(c1,c3)은 휴대폰 하고 있는 것으로 퉁쳐서 c1으로 클래스 구분하자
#                           -추가로 데이터 추가하고 클래스가 없는 too close(c5) / too far(c6)는 데이터 만들어야 겠네

delta = random.randint(0,10)
d =0

for pick, i in enumerate(Y_test):
    if ( (np.argmax(i)==2) and (np.argmax(X_pred[pick])== np.argmax(i))):
        d+=1
        if ( d> delta):
            print(X_pred[pick])
            break

heatmap = xai.explainable_model(X_test[pick], "2nd_relu", alpha=0.4)
fig_title = "Predicted {} as {:.2f}prob".format(np.argmax(X_pred[pick]) , np.max(X_pred[pick])) + ",  Label(GT): {}".format(np.argmax(Y_test[pick]))
plt.title(fig_title)
heatmap = xai.explainable_model(X_test[pick], "3rd_conv", alpha=0.6)
fig_title = "Predicted {} as {:.2f}prob".format(np.argmax(X_pred[pick]) , np.max(X_pred[pick])) + ",  Label(GT): {}".format(np.argmax(Y_test[pick]))
plt.title(fig_title)
heatmap = xai.explainable_model(X_test[pick], "3rd_maxpool", alpha=0.8)
fig_title = "Predicted {} as {:.2}fprob".format(np.argmax(X_pred[pick]) , np.max(X_pred[pick])) + ",  Label(GT): {}".format(np.argmax(Y_test[pick]))
plt.title(fig_title)


## 2. Just Inferencing without Label

In [7]:
### No label data(test)
from load.load_data import load_opendata

openloader = load_opendata()
X_open_test, Y_open_files = openloader.load_data(classifier_label=None, dsize=(64,64), comp_ratio=30)

print("My Test set(No Label) is ", len(X_open_test))
X_open_test = np.array(X_open_test)/255.0

Test opendata without Label: 100%|██████████| 79726/79726 [00:14<00:00, 5690.35it/s]

My Test set(No Label) is  2658


In [8]:
model_load.summary()

Model: "oop_classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
1st_conv (Conv2D)            (None, 64, 64, 64)        4864      
_________________________________________________________________
1st_relu (ReLU)              (None, 64, 64, 64)        0         
_________________________________________________________________
1st_maxpool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
2nd_conv (Conv2D)            (None, 32, 32, 128)       204928    
_________________________________________________________________
2nd_relu (ReLU)              (None, 32, 32, 128)       0         
_________________________________________________________________
2nd_maxpool (MaxPooling2D)   (None, 16, 16, 128)    

In [32]:
X_open_pred = model_load.predict(X_open_test[0:100])
print(X_open_pred.shape)

(100, 5)


In [35]:
np.sum(X_open_pred, axis=1)

array([1.        , 1.        , 1.        , 1.0000001 , 0.99999994,
       1.        , 1.        , 0.99999994, 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.99999994,
       1.0000001 , 1.        , 1.        , 0.99999994, 0.99999994,
       0.99999994, 1.        , 0.99999994, 1.        , 1.0000001 ,
       1.        , 1.        , 0.99999994, 0.99999994, 0.99999994,
       1.        , 0.99999994, 1.        , 1.        , 0.99999994,
       1.        , 0.99999994, 1.        , 0.99999994, 0.99999994,
       1.0000001 , 1.0000001 , 1.        , 1.        , 0.99999994,
       1.        , 1.        , 1.        , 1.        , 1.0000001 ,
       1.        , 1.        , 0.99999994, 1.        , 0.9999999 ,
       1.        , 0.9999999 , 1.0000001 , 1.        , 1.        ,
       0.99999994, 1.        , 1.        , 1.        , 0.99999994,
       1.        , 0.99999994, 1.0000001 , 1.        , 0.99999994,
       1.        , 1.        , 0.99999994, 0.99999994, 1.00000

In [22]:
np.sum(np.exp(X_open_pred[0]) /  np.sum(np.exp(X_open_pred[0])))

1.0

In [ ]:
# labels_map={'c5': 0, 'c0': 1, 'c6': 2, 'c1': 3, 'c7': 4}
#2. (모델2: OOP + phoning**) 기존 오픈데이터셋의 라벨링 활용 가능 safe driving(c0), reaching behind(c7)
#                           ,phoning(c2,c4), texting(c1,c3)은 휴대폰 하고 있는 것으로 퉁쳐서 c1으로 클래스 구분하자
#                           -추가로 데이터 추가하고 클래스가 없는 too close(c5) / too far(c6)는 데이터 만들어야 겠네
# pick = random.randint(0, len(X_open_test)-1)

delta = random.randint(0,100)
d =0
for pick, i in enumerate(X_open_pred):
    if  (np.argmax(i)==4):
        d+=1
        if ( d> delta):
            break

heatmap = xai.explainable_model(X_open_test[pick], "2nd_relu", alpha=0.4)
fig_title = "Predicted {} as {:.2f}prob".format(np.argmax(X_open_pred[pick]) , np.max(X_open_pred[pick]))
plt.title(fig_title)
heatmap = xai.explainable_model(X_open_test[pick], "3rd_conv", alpha=0.6)
fig_title = "Predicted {} as {:.2f}prob".format(np.argmax(X_open_pred[pick]) , np.max(X_open_pred[pick]))
plt.title(fig_title)
heatmap = xai.explainable_model(X_open_test[pick], "3rd_maxpool", alpha=0.8)
fig_title = "Predicted {} as {:.2}fprob".format(np.argmax(X_open_pred[pick]) , np.max(X_open_pred[pick]))
plt.title(fig_title)

In [ ]:
tf.keras.utils.plot_model("", show_shapes=True)